In [1]:
from ipywidgets import interact, widgets
interact(lambda x: x**2, x=widgets.IntSlider(min=0, max=10));

interactive(children=(IntSlider(value=0, description='x', max=10), Output()), _dom_classes=('widget-interact',…

# 🌍 Comparing Real GDP Across Countries: The PPP Adjustment

Comparing living standards across countries using GDP per capita requires more than just converting currencies at market exchange rates. Why? Because the **same amount of money buys different amounts of goods and services in different countries** due to variations in price levels.

For example, $1 might buy a coffee in one country but only half a coffee in another. Simply comparing GDP converted at the market exchange rate would understate the purchasing power (and thus the real living standard) in the lower-price country.

# ⚖️ Purchasing Power Parity (PPP)

**Purchasing Power Parity (PPP)** is a theory and a statistical adjustment that aims to equalize the purchasing power of different currencies. It calculates an "exchange rate" based on the relative prices of a common basket of goods and services in different countries.

**The Idea:** How much money would you need in Country B to buy the same basket of goods and services that $1 buys in Country A (often the US)? This ratio gives the PPP exchange rate.

**PPP Adjustment for GDP:** To make meaningful comparisons of real output or living standards, we adjust Nominal GDP per capita using PPP price level indexes:

$$ \text{Real GDP per capita (PPP adjusted)} = \frac{\text{Nominal GDP per capita (in local currency or USD)}}{\text{Price Level Index (relative to base country, e.g., US=1)}} $$

Or, more simply, if we have price levels relative to a base (like $P_{US}=1$ and $P_{India}=0.3$):

$$ \text{Real GDP}_{\text{India}}^{\text{PPP}} (\text{in US dollars}) = \frac{\text{Nominal GDP}_{\text{India}} (\text{in USD})}{P_{\text{India}}} $$

**Interpretation:**
* If a country's price level index is *less than 1* (meaning prices are lower than in the US), its PPP-adjusted GDP will be *higher* than its nominal GDP converted at market exchange rates.
* If a country's price level index is *greater than 1* (prices are higher than in the US), its PPP-adjusted GDP will be *lower*.

PPP adjustment provides a better comparison of the *volume* of goods and services people can actually consume in different countries.

In [2]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Layout
from IPython.display import display, Markdown
import warnings

# Optional: Use a specific style
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except IOError:
    pass # Use default if style not found

def ppp_gdp_comparison(
    gdp_cap_ctry1=60000.0, # e.g., US
    gdp_cap_ctry2=8000.0,  # e.g., India
    price_level_ctry1=1.0, # Base country, price level = 1
    price_level_ctry2=0.3  # Price level relative to base country
    ):
    """
    Compares Nominal GDP per capita with PPP-adjusted GDP per capita
    for two countries.

    Args:
        gdp_cap_ctry1 (float): Nominal GDP per capita (e.g., in USD) for Country 1.
        gdp_cap_ctry2 (float): Nominal GDP per capita (e.g., in USD) for Country 2.
        price_level_ctry1 (float): Price level index for Country 1 (Base=1).
        price_level_ctry2 (float): Price level index for Country 2 (relative to Base).
    """
    # Input validation
    price_level_ctry1 = max(price_level_ctry1, 1e-6) # Avoid division by zero
    price_level_ctry2 = max(price_level_ctry2, 1e-6)

    # Calculate PPP-adjusted GDP per capita
    # Real (PPP) = Nominal / Price Level Index
    ppp_gdp_ctry1 = gdp_cap_ctry1 / price_level_ctry1
    ppp_gdp_ctry2 = gdp_cap_ctry2 / price_level_ctry2

    # --- Prepare for Plotting ---
    countries = ['Country 1 (Base)', 'Country 2']
    nominal_gdp = np.array([gdp_cap_ctry1, gdp_cap_ctry2])
    ppp_gdp = np.array([ppp_gdp_ctry1, ppp_gdp_ctry2])

    # --- Plotting ---
    fig, ax = plt.subplots(figsize=(8, 6))
    x = np.arange(len(countries)) # Label locations
    width = 0.35 # Width of the bars

    bars1 = ax.bar(x - width/2, nominal_gdp, width, label='Nominal GDP per capita', color='#ff7f0e', alpha=0.8, edgecolor='black')
    bars2 = ax.bar(x + width/2, ppp_gdp, width, label='PPP-Adjusted GDP per capita', color='#1f77b4', alpha=0.8, edgecolor='black')

    # Add labels, title, and ticks
    ax.set_ylabel('GDP per capita ($)')
    ax.set_title('Nominal vs. PPP-Adjusted GDP per Capita Comparison')
    ax.set_xticks(x)
    ax.set_xticklabels(countries)
    ax.legend()
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    # Format y-axis
    ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda y, _: f'${y:,.0f}'))

    # Add bar labels
    ax.bar_label(bars1, fmt='$%.0f', padding=3, fontsize=9)
    ax.bar_label(bars2, fmt='$%.0f', padding=3, fontsize=9)

    fig.tight_layout()
    plt.show()

    # --- Display Ratios ---
    # Calculate ratios (handle division by zero)
    ratio_nominal = gdp_cap_ctry1 / gdp_cap_ctry2 if gdp_cap_ctry2 != 0 else np.inf
    ratio_ppp = ppp_gdp_ctry1 / ppp_gdp_ctry2 if ppp_gdp_ctry2 != 0 else np.inf

    results_md = f"""
    ### ⚖️ Comparison Summary:

    | Measure                     | Country 1 (Base) | Country 2        | Ratio (Ctry 1 / Ctry 2) |
    |-----------------------------|-----------------|------------------|-------------------------|
    | **Nominal GDP per capita** | ${gdp_cap_ctry1:,.0f}    | ${gdp_cap_ctry2:,.0f}     | {ratio_nominal:.1f} x             |
    | **Price Level Index** | {price_level_ctry1:.2f}         | {price_level_ctry2:.2f}          |                         |
    | **PPP-Adjusted GDP p.c.** | **${ppp_gdp_ctry1:,.0f}** | **${ppp_gdp_ctry2:,.0f}** | **{ratio_ppp:.1f} x** |

    *Note: PPP adjustment typically narrows the income gap between high-price (richer) and low-price (poorer) countries.*
    """
    display(Markdown(results_md))


# --- Create Interactive Widgets ---
style = {'description_width': 'initial'}
layout = Layout(width='95%')

interact(ppp_gdp_comparison,
         gdp_cap_ctry1=FloatSlider(value=60000.0, min=10000, max=120000, step=1000, description='Ctry 1 Nom GDP/cap ($):', style=style, layout=layout, readout_format=',.0f'),
         gdp_cap_ctry2=FloatSlider(value=8000.0, min=500, max=50000, step=500, description='Ctry 2 Nom GDP/cap ($):', style=style, layout=layout, readout_format=',.0f'),
         price_level_ctry1=FloatSlider(value=1.0, min=0.8, max=1.5, step=0.01, description='Ctry 1 Price Lvl (Base=1):', style=style, layout=layout, readout_format='.2f', disabled=True), # Keep Base=1
         price_level_ctry2=FloatSlider(value=0.3, min=0.1, max=1.5, step=0.01, description='Ctry 2 Price Lvl (Rel Base):', style=style, layout=layout, readout_format='.2f')
        );


interactive(children=(FloatSlider(value=60000.0, description='Ctry 1 Nom GDP/cap ($):', layout=Layout(width='9…

# 🌍 Real GDP Across Countries

Two countries might have:
- The same **output**
- But very **different prices**

So, we adjust GDP using **Purchasing Power Parity (PPP)**:
\[
\text{Real GDP per capita (PPP)} = \frac{\text{Nominal GDP}}{\text{Price Level Index}}
\]


# 💡 PPP Explained
If a burger costs:
- $1 in India
- $4 in the US  
...then $1 in India **buys 4x more stuff**.

So, $1 of GDP in India = $4 of PPP-adjusted GDP in the US


# 🔍 Why This Matters
Without PPP, we understate how much people in low-price countries can buy.  
**Living standards ≠ dollar GDP.**

This is crucial for comparing:
- China vs. US
- India vs. Mexico